In [1]:
import os
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
import scipy.constants as consts
import scipy.integrate as intg
import IPython.display as disp
import matplotlib.pyplot as plt
%load_ext autoreload 
%autoreload 2

# Utility Functions

In [43]:
def integrate(arr, layer_obj, sim_proc):
    arr_slice = arr[layer_obj.slice]
    z_vals = sim_proc.Z[layer_obj.istart:layer_obj.iend]
    #print("Layer: {}".format(layer_obj.name))
    #print("Layer Start Ind: {}".format(layer_obj.istart))
    #print("Layer End Ind: {}".format(layer_obj.iend))
    #print(z_vals)
    z_integral = intg.trapz(arr_slice, x=z_vals, axis=0)
    x_integral = intg.trapz(z_integral, x=sim_proc.X, axis=0)
    y_integral = intg.trapz(x_integral, x=sim_proc.Y, axis=0)
    return y_integral

def integrate_endpoints(arr, layer_obj, sim_proc):
    arr_slice = arr[layer_obj.slice]
    Nx = arr_slice.shape[1]
    Ny = arr_slice.shape[2]
    Nz = arr_slice.shape[0]
    extended = np.zeros((Nz, Nx+1, Ny+1), dtype=arr_slice.dtype)
    extended[:, 0:Nx, 0:Ny] = arr_slice
    extended[:, 0:Nx, -1] = arr_slice[:, 0:Nx, 0]
    extended[:, -1, 0:Ny] = arr_slice[:, 0, 0:Ny]
    extended[:, -1, -1] = arr_slice[:, 0, 0]
    zsamps = layer_obj.iend - layer_obj.istart
    z_vals = sim_proc.Z[layer_obj.istart:layer_obj.iend]
    x_vals = np.append(sim_proc.X, sim_proc.X[-1]+sim_proc.dx)
    y_vals = np.append(sim_proc.Y, sim_proc.Y[-1]+sim_proc.dy)
    z_integral = intg.trapz(extended, x=z_vals, axis=0)
    x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
    y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
    return y_integral

def compute_fluxes(sim):
    fluxes = sim.flux_dict
    total_incident_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][0])
    total_reflected_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][1])
    total_transmitted_power = .5*sim.period**2/Zo*np.absolute(sum(fluxes['Substrate_bottom']))
    total_absorbed_power = total_incident_power - total_reflected_power - total_transmitted_power
    print('Total Incident Power = {}'.format(total_incident_power))
    print('Total Reflected Power = {}'.format(total_reflected_power))
    print('Total Transmitted Power = {}'.format(total_transmitted_power))
    print('Total Absorbed Power = {}'.format(total_absorbed_power))
    summed_absorbed_power = 0
    abs_dict_fluxmethod = {}
    for layer, (forw_top, back_top) in fluxes.items():
        if '_bottom' in layer:
            continue
        bottom = layer+'_bottom'
        forw_bot, back_bot = fluxes[bottom] 
        print('-'*25)
        print('Layer: {}'.format(layer))
        print('Forward Top: {}'.format(forw_top))
        print('Backward Top: {}'.format(back_top))
        print('Forward Bottom: {}'.format(forw_bot))
        print('Backward Bottom: {}'.format(back_bot))
        P_in = forw_top + -1*back_bot
        P_out = forw_bot + -1*back_top
        print('Power Entering Layer: {}'.format(P_in))
        print('Power Leaving Layer: {}'.format(P_out))
        P_lost = P_in - P_out
        P_abs = .5*P_lost*(sim.period**2)/Zo
        abs_dict_fluxmethod[layer] = P_abs 
        print('Absorbed in Layer: {}'.format(P_abs))
        summed_absorbed_power += P_abs
    print('-'*25)
    print('Summed Absorption= {}'.format(summed_absorbed_power))
    return abs_dict_fluxmethod, summed_absorbed_power

# NW w/ Shell

## No Endpoints

In [44]:
conf = Config('AbsorptionTest.yml')
sim = Simulator(conf)
sim.setup()
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [45]:
sim.get_fluxes()

{'Air': ((930.4485521295919+0j), (-80.13269141567162+0j)),
 'Air_bottom': ((930.4485521295919+0j), (-80.13269141567159+0j)),
 'ITO': ((862.7587246914472-0.002097954481151021j),
  (-12.442863977490237+0.002097954481151021j)),
 'ITO_bottom': ((807.4789701136685-0.5135047176124203j),
  (-12.586816093866625+0.5135047176124203j)),
 'NW_AlShell': ((797.789901550103+0.15440143176019916j),
  (-2.8977475303095144-0.15440143176019916j)),
 'NW_AlShell_bottom': ((286.9173719835523+0.4969888559825204j),
  (-9.692889676537526-0.4969888559825204j)),
 'Substrate': ((277.2244823070163+0j), 0j),
 'Substrate_bottom': ((41.2276166626902+0j), 0j)}

In [46]:
abs_dict_fluxmethod, flux_method_total = compute_fluxes(sim)

Total Incident Power = 0.07718125198597892
Total Reflected Power = 0.006647053654188756
Total Transmitted Power = 0.0034198549324855635
Total Absorbed Power = 0.0671143433993046
-------------------------
Layer: Substrate
Forward Top: (277.2244823070163+0j)
Backward Top: 0j
Forward Bottom: (41.2276166626902+0j)
Backward Bottom: 0j
Power Entering Layer: (277.2244823070163+0j)
Power Leaving Layer: (41.2276166626902+0j)
Absorbed in Layer: (0.01957607813296811+0j)
-------------------------
Layer: Air
Forward Top: (930.4485521295919+0j)
Backward Top: (-80.13269141567162+0j)
Forward Bottom: (930.4485521295919+0j)
Backward Bottom: (-80.13269141567159+0j)
Power Entering Layer: (1010.5812435452635+0j)
Power Leaving Layer: (1010.5812435452635+0j)
Absorbed in Layer: 0j
-------------------------
Layer: ITO
Forward Top: (862.7587246914472-0.002097954481151021j)
Backward Top: (-12.442863977490237+0.002097954481151021j)
Forward Bottom: (807.4789701136685-0.5135047176124203j)
Backward Bottom: (-12.5868

In [47]:
sim.get_field()

In [48]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
int_method_total = 0
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)
    int_method_total += p_abs_imag
print("Integral Method Total Absorption: {}".format(int_method_total))

Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: Substrate


Integral Method Total Absorption: 0.06801109816284366


In [49]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'Air': 0j, 'Substrate': (0.01957607813296811+0j), 'NW_AlShell': (0.04294083635684372+0j), 'ITO': (0.004597428909495241+0j)}
{'Air': 0.0, 'ITO': 0.0045102648063681848, 'Substrate': 0.01927029314792518, 'NW_AlShell': 0.044230540208550294}


In [50]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))
pdiff_total = 100*abs(flux_method_total - int_method_total)/flux_method_total
print('-'*25)
print("Total Percent Difference: {}".format(pdiff_total))

-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None
-------------------------
Layer: Substrate
Flux Method: (0.01957607813296811+0j)
Integral Method: 0.01927029314792518
Diff: (0.0003057849850429291+0j)
Percent Diff: (1.5620339424777634+0j)
-------------------------
Layer: NW_AlShell
Flux Method: (0.04294083635684372+0j)
Integral Method: 0.044230540208550294
Diff: (-0.0012897038517065765+0j)
Percent Diff: (3.0034437172787607+0j)
-------------------------
Layer: ITO
Flux Method: (0.004597428909495241+0j)
Integral Method: 0.004510264806368185
Diff: (8.716410312705619e-05+0j)
Percent Diff: (1.8959315052601016+0j)
-------------------------
Total Percent Difference: (1.336159631632265+0j)


## Include Endpoints

In [51]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
int_method_total = 0
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate_endpoints(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)
    int_method_total += p_abs_imag
print("Integral Method Total Absorption: {}".format(int_method_total))

Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: Substrate


Integral Method Total Absorption: 0.06824292723795632


In [52]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'Air': 0j, 'Substrate': (0.01957607813296811+0j), 'NW_AlShell': (0.04294083635684372+0j), 'ITO': (0.004597428909495241+0j)}
{'Air': 0.0, 'ITO': 0.0045550194469211411, 'Substrate': 0.019457333658422827, 'NW_AlShell': 0.044230574132612346}


In [53]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))
pdiff_total = 100*abs(flux_method_total - int_method_total)/flux_method_total
print('-'*25)
print("Total Percent Difference: {}".format(pdiff_total))

-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None
-------------------------
Layer: Substrate
Flux Method: (0.01957607813296811+0j)
Integral Method: 0.019457333658422827
Diff: (0.00011874447454528203+0j)
Percent Diff: (0.6065794881831016+0j)
-------------------------
Layer: NW_AlShell
Flux Method: (0.04294083635684372+0j)
Integral Method: 0.044230574132612346
Diff: (-0.001289737775768629+0j)
Percent Diff: (3.003522719144888+0j)
-------------------------
Layer: ITO
Flux Method: (0.004597428909495241+0j)
Integral Method: 0.004555019446921141
Diff: (4.240946257409988e-05+0j)
Percent Diff: (0.9224604318842221+0j)
-------------------------
Total Percent Difference: (1.681583669729982+0j)


# NW w/ Shell w/ Lanczos

In [54]:
conf = Config('AbsorptionTest.yml')
conf[('Solver', 'LanczosSmoothing')] = True
sim = Simulator(conf)
sim.setup()
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

## No Endpoints

In [55]:
sim.get_fluxes()

{'Air': ((930.4485521295919+0j), (-78.6856892093137+0j)),
 'Air_bottom': ((930.4485521295919-7.105427357601002e-15j),
  (-78.68568920931372+7.105427357601002e-15j)),
 'ITO': ((857.6419173711085-0.08344942283687828j),
  (-5.879054450793626+0.08344942283687828j)),
 'ITO_bottom': ((802.3123460231939-0.280906236520142j),
  (-5.954610605004693+0.280906236520142j)),
 'NW_AlShell': ((800.4912303986539+0.582839589143596j),
  (-4.133494980474132-0.582839589143596j)),
 'NW_AlShell_bottom': ((313.3864735787744+0.84634589837402j),
  (-11.5054934474821-0.84634589837402j)),
 'Substrate': ((301.8809801312894+0j), 0j),
 'Substrate_bottom': ((45.148533189286084+0j), 0j)}

In [56]:
abs_dict_fluxmethod, flux_method_total = compute_fluxes(sim)

Total Incident Power = 0.07718125198597892
Total Reflected Power = 0.0065270239742482965
Total Transmitted Power = 0.003745097253259294
Total Absorbed Power = 0.06690913075847132
-------------------------
Layer: Substrate
Forward Top: (301.8809801312894+0j)
Backward Top: 0j
Forward Bottom: (45.148533189286084+0j)
Backward Bottom: 0j
Power Entering Layer: (301.8809801312894+0j)
Power Leaving Layer: (45.148533189286084+0j)
Absorbed in Layer: (0.02129610673804124+0j)
-------------------------
Layer: Air
Forward Top: (930.4485521295919+0j)
Backward Top: (-78.6856892093137+0j)
Forward Bottom: (930.4485521295919-7.105427357601002e-15j)
Backward Bottom: (-78.68568920931372+7.105427357601002e-15j)
Power Entering Layer: (1009.1342413389057-7.105427357601002e-15j)
Power Leaving Layer: (1009.1342413389057-7.105427357601002e-15j)
Absorbed in Layer: 0j
-------------------------
Layer: ITO
Forward Top: (857.6419173711085-0.08344942283687828j)
Backward Top: (-5.879054450793626+0.08344942283687828j)
F

In [57]:
sim.get_field()

In [58]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
int_method_total = 0
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)
    int_method_total += p_abs_imag
print("Integral Method Total Absorption: {}".format(int_method_total))

Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: Substrate


Integral Method Total Absorption: 0.06859024283346621


In [59]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'Air': 0j, 'Substrate': (0.02129610673804124+0j), 'NW_AlShell': (0.04101713626579001+0j), 'ITO': (0.004595887754642096+0j)}
{'Air': 0.0, 'ITO': 0.0045089416791430611, 'Substrate': 0.020964097336854379, 'NW_AlShell': 0.043117203817468773}


In [60]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))
pdiff_total = 100*abs(flux_method_total - int_method_total)/flux_method_total
print('-'*25)
print("Total Percent Difference: {}".format(pdiff_total))

-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None
-------------------------
Layer: Substrate
Flux Method: (0.02129610673804124+0j)
Integral Method: 0.02096409733685438
Diff: (0.0003320094011868627+0j)
Percent Diff: (1.5590145432253784+0j)
-------------------------
Layer: NW_AlShell
Flux Method: (0.04101713626579001+0j)
Integral Method: 0.04311720381746877
Diff: (-0.0021000675516787623+0j)
Percent Diff: (5.1199760462807+0j)
-------------------------
Layer: ITO
Flux Method: (0.004595887754642096+0j)
Integral Method: 0.004508941679143061
Diff: (8.694607549903505e-05+0j)
Percent Diff: (1.8918233024993873+0j)
-------------------------
Total Percent Difference: (2.512530137420687+0j)


## Include Endpoints

In [61]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
int_method_total = 0
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate_endpoints(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)
    int_method_total += p_abs_imag
print("Integral Method Total Absorption: {}".format(int_method_total))

Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: Substrate


Integral Method Total Absorption: 0.0688385495300369


In [62]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'Air': 0j, 'Substrate': (0.02129610673804124+0j), 'NW_AlShell': (0.04101713626579001+0j), 'ITO': (0.004595887754642096+0j)}
{'Air': 0.0, 'ITO': 0.0045537500761615264, 'Substrate': 0.021167558156111103, 'NW_AlShell': 0.043117241297764279}


In [63]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))
pdiff_total = 100*abs(flux_method_total - int_method_total)/flux_method_total
print('-'*25)
print("Total Percent Difference: {}".format(pdiff_total))

-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None
-------------------------
Layer: Substrate
Flux Method: (0.02129610673804124+0j)
Integral Method: 0.021167558156111103
Diff: (0.00012854858193013866+0j)
Percent Diff: (0.60362480105583+0j)
-------------------------
Layer: NW_AlShell
Flux Method: (0.04101713626579001+0j)
Integral Method: 0.04311724129776428
Diff: (-0.0021001050319742684+0j)
Percent Diff: (5.120067423443803+0j)
-------------------------
Layer: ITO
Flux Method: (0.004595887754642096+0j)
Integral Method: 0.004553750076161526
Diff: (4.213767848056972e-05+0j)
Percent Diff: (0.9168561272630816+0j)
-------------------------
Total Percent Difference: (2.8836404683365435+0j)


# NW no Shell

In [ ]:
conf = Config('AbsorptionTestnoshell.yml')
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [ ]:
sim.get_fluxes()

In [ ]:
abs_dict_fluxmethod = compute_fluxes(sim)

In [ ]:
sim.get_field()

In [ ]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

In [ ]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

In [ ]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

In [ ]:
print(sim.data['fluxes'])

In [ ]:
print(sim.flux_dict)

In [ ]:
simulation = Simulation(simulator=sim)
print(simulation.data['fluxes'])
print(simulation.layers)

In [ ]:
alist = [1., 2., 3.]
print('{}, {}, {}, {}'.format('test', *alist))